In [1]:
!pip install --upgrade pip
!pip install transformers==4.40.1 datasets==2.19.0 sentence-transformers==2.7.0 faiss-cpu==1.8.0 llama-index==0.10.34 llama-index-embeddings-huggingface==0.2.0 -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s e

# **문장 임베딩을 활용한 단어 간 유사도 계산**

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

smodel = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
dense_embeddings = smodel.encode(['학교', '공부', '운동'])
cosine_similarity(dense_embeddings) #코사인 유사도
# array([[1.0000001 , 0.5950744 , 0.32537547],
#       [0.5950744 , 1.0000002 , 0.54595673],
#       [0.32537547, 0.54595673, 0.99999976]], dtype=float32)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


array([[0.99999994, 0.59507436, 0.32537556],
       [0.59507436, 1.        , 0.54595697],
       [0.32537556, 0.54595697, 0.99999994]], dtype=float32)

# **원핫 인코딩의 한계**

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

word_dict = {"school": np.array([[1, 0, 0]]), "study": np.array([[0, 1, 0]]), "workout": np.array([[0, 0, 1]])}

# 두 단어 사이의 코사인 유사도 계산하기
# cosine_similarity(A, B) -> 코사인 유사도 측정 메서드
cosine_school_study = cosine_similarity(word_dict['school'], word_dict['study']) # 0
cosine_school_workout = cosine_similarity(word_dict['school'], word_dict['workout']) # 0

# **Sentence-Transformers 라이브러리로 바이 인코더 생성하기**

In [ ]:
# 문장을 벡터로 변환하는 라이브러리
from sentence_transformers import SentenceTransformer, models

# 사용할 BERT 모델
# 허깅페이스에서 임베딩 모델 불러오기
word_embedding_model = models.Transformer('klue/roberta-base')
# 풀링 층 차원 입력하기 -> 단어별 임베딩을 하나의 문장 임베딩으로 압축
# get_word_embedding_dimension() -> 모델의 임베딩 크기(차원 수)를 가져옴
# 단어 차원의 수를 가져와서 문장 벡터의 차원으로 설정
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# 두 모듈 결합하기
# 임베딩 모델과 풀링층 결합
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

# **코드로 살펴보는 평균 모드**

In [ ]:
def mean_pooling(model_output, attention_mask):
  # model_output -> 언어 모델 출력
  # model_output[0] -> 출력 중 마지막 층의 출력만 사용 -> 마지막 층에서 나온 토큰별 임베딩(벡터) -> 모든 처리를 거쳐 [0]에 저장되기 때문
  # 0은 출력 리스트의 첫 번째 요소를 의미하는 것이지 첫 번째 층의 출력을 의미하는게 아님
  token_embeddings = model_output[0]
  # unsqueeze(-1) -> 마지막 차원 추가
  # attention_mask는 [batch_size, num_tokens] 형태라서 token_embeddings(batch_size, num_tokens, hidden_dim)와 곱할 수 없다

  # expand(token_embeddings.size()) -> 텐서 차원 확장 -> token_embeddings.size()와 같게

  # .float() -> 벡터와 곱하기 위해 형변환
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

  # attention_mask -> 미래 마스크가 아닌 패딩 마스크
  # 패딩 토큰이 학습과 계산에 영향을 주지 않도록 무시하는 역할
  # BERT 같은 인코더 기반 모델에서 사용

  # 패딩이 0이 아닐수도 있어서 패딩 마스크와 곱해 패딩 부분의 값을 0으로 만든다
  # sum(x, 1) -> 두번째 차원을 따라 더한다
  sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

  # input_mask_expanded.sum(1) -> 패딩 마스크의 두 번째 차원을 더해 단어 개수를 구한다
  # input_mask_expanded =
    # [[1, 1, 1, 0, 0],  # 첫 번째 문장 (5개 토큰 중 3개가 실제 단어)
    # [1, 1, 1, 1, 1]]  # 두 번째 문장 (모든 토큰이 실제 단어)

  # torch.clamp -> min 혹은 max의 범주에 해당하도록 값 변경
  # min=1e-9 -> 최솟값이 해당 값보다 작으면 해당 값으로 변경
  # 0으로 나누는 것을 방지하기 위함
  sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

  return sum_embeddings / sum_mask

  # 1. 모델 아웃풋과 패딩 마스크를 받아
  # 2. 패딩 마스크를 모델 아웃풋과 곱할 수 있도록 차원을 바꾸고 형변환
  # 3. 모델 아웃풋과 패딩 마스크를 곱해 패딩 토큰을 0으로 만들고
  # 4. 모델 아웃풋을 모두 더한다
  # 5. 평균을 구하기 위해 패딩 마스크를 더해 단어 개수를 뽑고 (0으로 나누지 않기 위해서 최솟값 부여)
  # 6. 더한 출력 임베딩의 합을 단어 개수로 나눈다

# **코드로 살펴보는 최대 모드**

In [ ]:
def max_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

  # 텐서 조건 인덱싱
  # x = torch.tensor([10, 20, 30, 40, 50])
  # x[x > 25] = -1  # 25보다 큰 값만 -1로 변경
  # print(x)

  # token_embeddings와 input_mask_expanded의 차원이 같기 때문에
  # input_mask_expanded를 순회하면서 0인 인덱스를 찾아 token_embeddings에 적용
  # 이후 -1e9 할당
  # 즉 패딩 토큰이 최댓값이 절대 될 수 없도록 변경
  token_embeddings[input_mask_expanded == 0] = -1e9

  # 최댓값 반환
  # [0] = 최댓값
  # [1] = 최댓값이 있는 위치
  return torch.max(token_embeddings, 1)[0]

# **한국어 문장 임베딩 모델로 입력 문장 사이의 유사도 계산**

In [ ]:
from sentence_transformers import SentenceTransformer, util

# 바이 인코더
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 임베딩 벡터로 변환
embs = model.encode(['잠이 안 옵니다',
                     '졸음이 옵니다',
                     '기차가 옵니다'])

# embs와 embs를 비교하여 각 문장 간 코사인 유사도 행렬을 만듦
cos_scores = util.cos_sim(embs, embs)
print(cos_scores)
# 자기 자신과의 비교는 1.0 즉 100% 유사도
# tensor([[1.0000, 0.6410, 0.1887],
#         [0.6410, 1.0000, 0.2730],
#         [0.1887, 0.2730, 1.0000]])

# **CLIP 모델을 활용한 이미지와 텍스트 임베딩 유사도 계산**

In [ ]:
from PIL import Image
from sentence_transformers import SentenceTransformer, util

# CLIP (Contrastive Language-Image Pretraining)
# 텍스트와 이미지를 동시에 학습해서 이미지와 텍스트 간 의미적 유사도를 비교할 수 있는 모델
model = SentenceTransformer('clip-Vit-B-32')

# image.open('dog.jpg') -> 파일을 열어 PIL 이미지 객체로 변환
# 임베딩 변환
img_embs = model.encode([image.open('dog.jpg'), image.open('cat.jpg')])
text_embs = model.encode(['A dog on grass', 'Brown cat on yellow background'])

cos_scores = util.cos_sim(image_embs, text_embs)
print(cos_scores)
# tensor([[0.2771, 0.1509],
#         [0.2071, 0.3180]])

# **실습에 사용할 모델과 데이터셋 불러오기**

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **실습 데이터에서 1000개만 선택하고 문장 임베딩으로 변환**

In [3]:
klue_mrc_dataset = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']
embeddings = sentence_model.encode(klue_mrc_dataset['context'])
embeddings.shape
# 출력 결과
# (1000, 768)

(1000, 768)

# **KNN 검색 인덱스를 생성하고 문장 임베딩 저장**

In [4]:
import faiss

# 인덱스 만들기
# L2 거리(유클리드 거리) 기반 인덱스 생성
# embeddings.shape[1] 임베딩의 벡터 차원만큼 저장공간 생성
index = faiss.IndexFlatL2(embeddings.shape[1])

# 인덱스에 임베딩 저장하기
index.add(embeddings)

# **의미 검색의 장점**

In [5]:
query = "이번 연도에는 언제 비가 많이 올까?"
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
  # 처음 50글자만 출력
  print(klue_mrc_dataset['context'][idx][:50])

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그


# **의미 검색의 한계**

In [ ]:
query = klue_mrc_dataset[3]['question'] # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
query_embedding = sentence_model.encode([query])

distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 (오답)
# 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 (오답)
# 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스 (정답)

# **라마인덱스에서 Sentence-Transformers 임베딩 모델 활용**

In [ ]:
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 라마 인덱스에 등록하려면 해당 방법으로 로드
embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

# 로컬 모델 활용하기
# service_context = ServiceContext.from_defaults(embed_model="local")

text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index_llama = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context
)

# **BM25 클래스 구현**

In [ ]:
import math
import numpy as np
from typing import List
from transformers import PreTrainedTokenizer
from collections import defaultdict

class BM25:
  # corpus:List[List[str]] -> 타입 힌트
  # corpus -> 검색할 문서 리스트
  # tokenizer -> 문장을 토큰으로 변환할 토크나이저
  def __init__(self, corpus:List[List[str]], tokenizer:PreTrainedTokenizer):
    self.tokenizer = tokenizer
    self.corpus = corpus
    # 토크나이저로 문서를 토큰화해 ID 리스트를 할당
    # add_special_tokens=False는 스페셜 토큰([CLS], [SEP])을 추가하지 않겠다는 옵션
    self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
    # 문서 개수
    self.n_docs = len(self.tokenized_corpus)
    # 문서의 평균 길이
    self.avg_doc_lens = sum(len(lst) for lst in self.tokenized_corpus) / len(self.tokenized_corpus)
    self.idf = self._calculate_idf()
    self.term_freqs = self._calculate_term_freqs()

  # 각 토큰이 몇 개의 문서에 등장하는 집계하는 메서드
  def _calculate_idf(self):
    # 값이 없는 키를 조회할 때 기본값을 자동으로 넣어주는 딕셔너리 생성
    # float면 0.0이 기본값으로 들어감
    idf = defaultdict(float)

    # 문서 순회
    for doc in self.tokenized_corpus:
      # set(doc) -> 중복 제거
      # 즉 중복 단어 제거 / 문서에 등장 하는지 안하는지만 카운트
      for token_id in set(doc):
        idf[token_id] += 1
    # idf.items() -> 딕셔너리의 키-값 쌍을 가져오는 메서드
    for token_id, doc_frequency in idf.items():
      idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)
    return idf

  # 토큰이 각 문서 내에서 몇 번 등장하는 카운트
  def _calculate_term_freqs(self):
    term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
    for i, doc in enumerate(self.tokenized_corpus):
      for token_id in doc:
        term_freqs[i][token_id] += 1
    return term_freqs

  # 점수 계산
  def get_scores(self, query:str, k1:float = 1.2, b:float=0.75):
    query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
    # np.zeros -> 0으로 채운 배열 생성
    scores = np.zeros(self.n_docs)
    for q in query:
      idf = self.idf[q]
      for i, term_freq in enumerate(self.term_freqs):
        q_frequency = term_freq[q]
        doc_len = len(self.tokenized_corpus[i])
        score_q = idf * (q_frequency * (k1 + 1)) / ((q_frequency) + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
        scores[i] += score_q
    return scores

  # 상위 k개의 점수와 인덱스를 추출
  def get_top_k(self, query:str, k:int):
    scores = self.get_scores(query)
    # argsort -> 작은 것에서 큰 순서대로 정렬
    top_k_indices = np.argsort(scores)[-k:][::-1]
    top_k_scores = scores[top_k_indices]
    return top_k_scores, top_k_indices

# **BM25 점수 계산 확인해 보기**

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

bm25 = BM25(['안녕하세요', '반갑습니다', '안녕 서울'], tokenizer)
bm25.get_scores('안녕')
# array([0.44713859, 0.        , 0.52354835])

# **BM25 검색 결과의 한계**

In [ ]:
# BM25 검색 준비
bm25 = BM25(klue_mrc_dataset['context'], tokenizer)

query = "이번 연도에는 언제 비가 많이 올까?"
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다 (오답)
# 인구 비율당 노벨상을 세계에서 가장 많이 받은 나라, 과학 논문을 가장 많이 쓰고 의료 특 (오답)
# 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은  (정답)

# **BM25 검색 결과의 장점**

In [ ]:
query = klue_mrc_dataset[3]['question'] # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스 (정답)
# ;메카동(メカドン)                                                      (오답)
# :성우 : 나라하시 미키(ならはしみき)
# 길가에 버려져 있던 낡은 느티나
# ;메카동(メカドン)                                                      (오답)
# :성우 : 나라하시 미키(ならはしみき)
# 길가에 버려져 있던 낡은 느티나